<a href="https://colab.research.google.com/github/Signed-B/build-your-own-llm/blob/main/colab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q transformers accelerate datasets gradio einops evaluate
# !pip install -q peft
# !pip install -q trl xformers wandb sentencepiece bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 741.1 kB/s eta 0:00:00


In [7]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
# from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
# import wandb
import os,torch, platform, gradio, warnings
from datasets import load_dataset
# from trl import SFTTrainer
# from huggingface_hub import notebook_login

In [8]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: Tesla T4
Compute Capability: (7, 5)
Total Memory: 15835660288 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 6.1.58+
Python Version: 3.10.12


In [9]:
# Pre trained model
model_name = "mosaicml/mpt-7b"

# Dataset name
dataset_name = "vicgalle/alpaca-gpt4"

In [92]:
model = transformers.AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map='cuda',
  torch_dtype=torch.bfloat16,
  trust_remote_code=True
)

/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b/67cf22a4e6809edb7308dd0a2ae2c1ffb86f4984/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b/67cf22a4e6809edb7308dd0a2ae2c1ffb86f4984/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 394.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 101.06 MiB is free. Process 2157 has 14.65 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 17.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [44]:
dataset = load_dataset(dataset_name, split="train[0:1000]")
dataset = dataset.train_test_split(test_size=0.2, seed=42)
dataset
# TODO increase number of rows.

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 800
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 200
    })
})

In [45]:
print(dataset['train']['text'][0])
print("-------")
print(dataset['train']['text'][1])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Generate a creative activity for a child to do during their summer vacation.

### Response:
One creative activity for a child to do during their summer vacation is to create a nature journal. To start, they will need a blank notebook or sketchbook and some art supplies such as colored pencils, crayons or watercolors. Encourage the child to go outside and explore their surroundings. They can take a walk in the park, go for a hike or just explore their own backyard.

Ask the child to observe the plants, animals, and insects they see along the way. They can sketch and write about what they see in their nature journal. This not only promotes creativity, but also helps the child to learn more about their environment and the natural world around them.

They can also collect leaves, flowers, or other small objects and glue them into their journal to create a natural coll

In [46]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token

def preprocess_function(examples):
  return tokenizer(examples['text'])

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset['train'].column_names
)
tokenized_dataset

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 800
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 200
    })
})

In [67]:
block_size = 128

def group_texts(examples):

  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])

  # remove remainder
  if total_length >= block_size:
    total_length = (total_length // block_size) * block_size

  result = {
      k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
      for k, t in concatenated_examples.items()
  }

  result["labels"] = result["input_ids"].copy()
  return result

final_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)
final_dataset

Map (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1177
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 295
    })
})

In [69]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [80]:
from transformers import Trainer

training_arguments = TrainingArguments(
    output_dir="output_dir",
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False
)
# TODO theory on each of the hyperparameters.

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["test"],
    data_collator=data_collator,
)

In [81]:
trainer.train()

/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b/67cf22a4e6809edb7308dd0a2ae2c1ffb86f4984/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 101.06 MiB is free. Process 2157 has 14.65 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 17.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [91]:
# del model
# del trainer
# del training_arguments
del final_dataset
torch.cuda.empty_cache()